In [1]:
%matplotlib inline
import utils#; reload(utils)
from utils import *
from __future__ import division, print_function


import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
import cv2 

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [6]:
ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data/'

source_path = DATA_HOME_DIR+"cropped_nof_excl/train_fullsize/"
export_path = DATA_HOME_DIR+'cropped_nof_excl/train/'

crop_width = 450

anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']


#### make validation subdirectories

In [7]:
# % cd data/nof_excl/train
# g = glob('*')
# for d in g: 
#     os.mkdir('../../cropped_nof_excl/train/'+d)
# #     os.mkdir('../../cropped_nof_excl/valid/'+d)

In [8]:
#load annotations

bb_json = {}

j = json.load(open('{}/all_fish_labels.json'.format("data"), 'r'))

for  i , l in enumerate(j):
    filename = l['filename'].split('/')[-1]
    if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[filename] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]

    if len(l['annotations']) ==0:
        bb_json[filename]  = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}


In [9]:
len(bb_json)

3312

In [10]:
for folder in glob(os.path.join(source_path, '*')):
    
       
    for f in glob(os.path.join(folder, '*')):
        img = cv2.imread(f)
#         plt.imshow(img)
        img_width = img.shape[1]
        img_height = img.shape[0]
        
        
        bb = bb_json[f.split('/')[-1]]
        x = bb['x'] #bottom left
        y = bb['y']
          
        
        #find center of the box

        cX = x + bb['width']/2
        cY = y + bb['height']/2


        #calculate boundries of bounding box (overriding x,y to make bbox bigger)
        x = max(cX - crop_width/2, 0)
        y = max(cY - crop_width/2, 0)
      
        if x + crop_width > img_width:
            x = img_width - crop_width
        if y + crop_width > img_height:
            y = img_height - crop_width

        
        x = int(x)
        y = int(y)
        
##### plot bounding box    
#         cv2.rectangle(img,(x,y),(x+crop_width,y+crop_width),color=(255, 255, 255))
#         plt.imshow(img)

            
    ##### crop the image and save
        crop_img = img[y:y+crop_width, x:x+crop_width]
#         plt.imshow(crop_img)
        
        foldername = folder.split('/')[-1]
        filename = f.split('/')[-1]
#         print(export_path+foldername+"/"+filename)
        cv2.imwrite(export_path+foldername+"/"+filename,crop_img)    
#         break